# Camera Projection Mathematics Demo

This notebook demonstrates how to project 3D world points to 2D image coordinates using camera parameters.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mattloper/robot-junkyard/blob/main/camera_projection_demo.ipynb)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import json
import requests
from PIL import Image
from io import BytesIO

In [ ]:
# Load the actual image from GitHub
img_url = 'https://raw.githubusercontent.com/mattloper/robot-junkyard/main/frame00070.png'
response = requests.get(img_url)
img = Image.open(BytesIO(response.content))
img_array = np.array(img)

print(f"Loaded image shape: {img_array.shape}")
plt.figure(figsize=(6, 10))
plt.imshow(img_array)
plt.title('Original frame00070.png')
plt.axis('off')
plt.show()

## 1. Camera Parameters

Let's start with the camera parameters from our first reference frame:

In [ ]:
# Camera data for frame00070.jpg
camera_data = {
    "filename": "frame00070.jpg",
    "width": 1083,
    "height": 1933,
    "w2c_rotation": [
        [-0.017509, -0.120612, -0.992545],
        [-0.979081, -0.199198,  0.041478],
        [-0.202716,  0.972509, -0.114601]
    ],
    "w2c_translation": [0.292837, 0.502510, 1.476158],
    "intrinsics": {
        "fx": 1609.799,
        "fy": 1611.112,
        "cx": 541.5,
        "cy": 966.5
    },
    "camera_position": [0.796366, -1.300158, 0.438980]
}

# Extract parameters
width = camera_data["width"]
height = camera_data["height"]
R = np.array(camera_data["w2c_rotation"])
t = np.array(camera_data["w2c_translation"]).reshape(3, 1)
fx = camera_data["intrinsics"]["fx"]
fy = camera_data["intrinsics"]["fy"]
cx = camera_data["intrinsics"]["cx"]
cy = camera_data["intrinsics"]["cy"]

# Build intrinsic matrix K
K = np.array([
    [fx,  0, cx],
    [ 0, fy, cy],
    [ 0,  0,  1]
])

print(f"Image size: {width}×{height}")
print(f"\nIntrinsic matrix K:")
print(K)
print(f"\nRotation matrix R (world-to-camera):")
print(R)
print(f"\nTranslation vector t:")
print(t.flatten())
print(f"\nCamera position in world: {camera_data['camera_position']}")

## 2. Define Test Points

We'll project the origin and three unit vectors along the axes:

In [ ]:
# Define canonical world points
world_points = [
    {"name": "Origin", "point": [0, 0, 0], "color": "black"},
    {"name": "X-axis", "point": [1, 0, 0], "color": "red"},
    {"name": "Y-axis", "point": [0, 1, 0], "color": "green"},
    {"name": "Z-axis", "point": [0, 0, 1], "color": "blue"}
]

for wp in world_points:
    print(f"{wp['name']}: {wp['point']}")

## 3. Projection Pipeline

The projection follows these steps:
1. **World to Camera**: Transform world coordinates to camera coordinates
2. **Visibility Check**: Ensure point is in front of camera (z > 0)
3. **Normalize**: Project to normalized image plane
4. **Apply Intrinsics**: Convert to pixel coordinates

In [ ]:
def project_point(p_world, R, t, K, width, height):
    """
    Project a 3D world point to 2D image coordinates.
    
    Returns:
        pixel_coords: (u, v) pixel coordinates
        is_visible: True if point is in front of camera
        is_in_bounds: True if pixel is within image bounds
        intermediate_results: dict with intermediate calculations
    """
    # Step 1: World to camera coordinates
    p_world = np.array(p_world).reshape(3, 1)
    p_cam = R @ p_world + t
    
    # Step 2: Check visibility
    z_cam = p_cam[2, 0]
    is_visible = z_cam > 0
    
    if is_visible:
        # Step 3: Project to normalized image plane
        p_norm = p_cam[:2, 0] / z_cam
        
        # Step 4: Apply intrinsics
        p_homogeneous = np.array([p_norm[0], p_norm[1], 1.0])
        p_pixel_homogeneous = K @ p_homogeneous
        u = p_pixel_homogeneous[0]
        v = p_pixel_homogeneous[1]
    else:
        # Point behind camera
        p_norm = np.array([0, 0])
        u, v = -1, -1
    
    # Check if in image bounds
    is_in_bounds = (0 <= u < width) and (0 <= v < height)
    
    return (u, v), is_visible, is_in_bounds, {
        "p_cam": p_cam.flatten(),
        "p_norm": p_norm,
        "z_cam": z_cam
    }

## 4. Project All Points

Now let's project each of our test points and see the detailed calculations:

In [ ]:
# Project all points and collect results
projection_results = []

for wp in world_points:
    name = wp["name"]
    p_world = wp["point"]
    color = wp["color"]
    
    pixel_coords, is_visible, is_in_bounds, intermediates = project_point(
        p_world, R, t, K, width, height
    )
    
    result = {
        "name": name,
        "color": color,
        "p_world": p_world,
        "pixel_coords": pixel_coords,
        "is_visible": is_visible,
        "is_in_bounds": is_in_bounds,
        **intermediates
    }
    projection_results.append(result)
    
    # Print detailed results
    print(f"\n{'='*60}")
    print(f"{name} Point: {p_world}")
    print(f"\nStep 1: World to camera")
    print(f"  p_cam = R @ p_world + t = {intermediates['p_cam']}")
    print(f"  z_cam = {intermediates['z_cam']:.6f}")
    
    if is_visible:
        print(f"\nStep 2: Normalize (divide by z_cam)")
        print(f"  p_norm = [{intermediates['p_norm'][0]:.6f}, {intermediates['p_norm'][1]:.6f}]")
        print(f"\nStep 3: Apply intrinsics")
        print(f"  u = fx * x_norm + cx = {fx:.1f} * {intermediates['p_norm'][0]:.6f} + {cx:.1f} = {pixel_coords[0]:.1f}")
        print(f"  v = fy * y_norm + cy = {fy:.1f} * {intermediates['p_norm'][1]:.6f} + {cy:.1f} = {pixel_coords[1]:.1f}")
        print(f"\nFinal pixel: ({pixel_coords[0]:.1f}, {pixel_coords[1]:.1f})")
        print(f"In bounds: {is_in_bounds}")
    else:
        print(f"\n⚠️  Point is BEHIND camera (z_cam < 0)")

# Create visualization with actual image
fig, ax = plt.subplots(1, 1, figsize=(8, 14))

# Display the actual image as background
ax.imshow(img_array, extent=[0, width, height, 0])

# Draw image boundary
ax.add_patch(plt.Rectangle((0, 0), width, height, fill=False, edgecolor='yellow', linewidth=2))

# Draw principal point
ax.plot(cx, cy, 'r+', markersize=20, markeredgewidth=3, label='Principal point')

# Plot projected points
for result in projection_results:
    if result["is_visible"]:
        u, v = result["pixel_coords"]
        # Larger markers with white outline for visibility
        ax.plot(u, v, 'o', color=result["color"], markersize=15, 
                markeredgecolor='white', markeredgewidth=3,
                label=f"{result['name']} ({u:.0f}, {v:.0f})")
        
        # Add text label with better visibility
        ax.text(u + 15, v - 15, result["name"], fontsize=12, weight='bold',
                bbox=dict(boxstyle="round,pad=0.5", facecolor='white', 
                         edgecolor=result["color"], linewidth=2, alpha=0.9))

# Set axis properties
ax.set_xlim(0, width)
ax.set_ylim(height, 0)  # Invert Y axis (image convention)
ax.set_aspect('equal')
ax.set_xlabel('u (pixels)', fontsize=12)
ax.set_ylabel('v (pixels)', fontsize=12)
ax.set_title(f'Projected Points on Actual Image ({width}×{height})', fontsize=16)
ax.legend(loc='upper right', fontsize=10, framealpha=0.9)

plt.tight_layout()
plt.show()

In [ ]:
# Create visualization
fig, ax = plt.subplots(1, 1, figsize=(8, 14))

# Draw image boundary
ax.add_patch(plt.Rectangle((0, 0), width, height, fill=False, edgecolor='black', linewidth=2))

# Draw principal point
ax.plot(cx, cy, 'r+', markersize=15, markeredgewidth=2, label='Principal point')

# Plot projected points
for result in projection_results:
    if result["is_visible"]:
        u, v = result["pixel_coords"]
        ax.plot(u, v, 'o', color=result["color"], markersize=10, 
                markeredgecolor='black', markeredgewidth=1,
                label=f"{result['name']} ({u:.0f}, {v:.0f})")
        
        # Add text label
        ax.text(u + 10, v - 10, result["name"], fontsize=10, 
                bbox=dict(boxstyle="round,pad=0.3", facecolor=result["color"], alpha=0.3))

# Set axis properties
ax.set_xlim(-50, width + 50)
ax.set_ylim(height + 50, -50)  # Invert Y axis (image convention)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.set_xlabel('u (pixels)')
ax.set_ylabel('v (pixels)')
ax.set_title(f'Projected Points in Image Frame ({width}×{height})', fontsize=14)
ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

## 6. Summary Table

Let's create a summary table of all projections:

In [ ]:
# Create summary table
print("\nProjection Summary:")
print("="*80)
print(f"{'Point':<10} {'World Coords':<20} {'Camera Z':<12} {'Pixel (u,v)':<20} {'Status':<20}")
print("="*80)

for result in projection_results:
    world_str = str(result['p_world'])
    z_str = f"{result['z_cam']:.3f}"
    
    if result['is_visible']:
        pixel_str = f"({result['pixel_coords'][0]:.1f}, {result['pixel_coords'][1]:.1f})"
        if result['is_in_bounds']:
            status = "✓ Visible & in bounds"
        else:
            status = "⚠ Visible but outside"
    else:
        pixel_str = "N/A"
        status = "✗ Behind camera"
    
    print(f"{result['name']:<10} {world_str:<20} {z_str:<12} {pixel_str:<20} {status:<20}")

## 7. Building a Perspective Projection Matrix (Optional)

If you need to build a 4×4 perspective projection matrix from these parameters:

In [ ]:
def build_projection_matrix(fx, fy, cx, cy, width, height, near=0.1, far=1000.0):
    """
    Build a 4x4 perspective projection matrix from intrinsic parameters.
    Uses OpenGL/COLMAP convention (right-handed, camera looks along +Z).
    """
    # Off-axis frustum boundaries
    l = -near * cx / fx
    r = near * (width - cx) / fx
    t = near * cy / fy  # Top is at cy (positive y goes down)
    b = -near * (height - cy) / fy  # Bottom is negative
    
    # Perspective projection matrix
    P = np.array([
        [2*near/(r-l),           0,  (l+r)/(r-l),                    0],
        [          0, 2*near/(t-b),  (t+b)/(t-b),                    0],
        [          0,           0, -(far+near)/(far-near), -2*far*near/(far-near)],
        [          0,           0,           -1,                    0]
    ])
    
    return P

# Build and display the projection matrix
P = build_projection_matrix(fx, fy, cx, cy, width, height)
print("4×4 Perspective Projection Matrix:")
print(P)